### **Customer**


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [0]:
import os 
import sys

In [0]:
current_dir = os.getcwd()
# current_dir
sys.path.append(current_dir)

In [0]:
df_cust = spark.read.table("pysparkdbt.bronze.customers")

In [0]:
display(df_cust)

In [0]:
df_cust = df_cust.withColumn("domain",split(col('email'),"@")[1])
display(df_cust)

In [0]:
df_cust = df_cust.withColumn("phone_number", regexp_replace(col("phone_number"), "[^0-9]", ""))
# display(df_cust)

In [0]:
df_cust =df_cust.withColumn("full_name",concat_ws(" ",col("first_name"),col("last_name")))
df_cust = df_cust.drop("first_name","last_name")
display(df_cust)

In [0]:
from utils.custom_utils import var_1
print(var_1)
